# Dataset Structure Analysis: LEGOS Sea Ice Thickness (SIT) Product

**Purpose:** Comprehensive structural inspection of the LEGOS SIT dataset for quality assurance and data validation.

**Dataset:** `SIT_SH_2002_2011_ENV_SnowAMSR.ease2_12500_smth25000.nc`

**Author:** Xinlong Liu  
**Last Updated:** 2025-12-07

In [1]:
"""
NetCDF Dataset Structure Inspector
==================================
This module provides comprehensive analysis of NetCDF file structure,
following Google Python Style Guide and Amazon's operational excellence principles.

Features:
- Complete metadata extraction
- Variable inspection with dimensions and attributes
- Data type and shape analysis
- Memory footprint estimation
"""

from __future__ import annotations

import os
import logging
from pathlib import Path
from typing import Any

import numpy as np
import xarray as xr

# Configure logging following Google's best practices
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [2]:
# Configuration Constants
# Following Amazon's principle of externalized configuration

DATA_DIR: Path = Path(r"D:\phd\data\LEGOS")
FILENAME: str = "SIT_SH_2002_2011_ENV_SnowAMSR.ease2_12500_smth25000.nc"
FILEPATH: Path = DATA_DIR / FILENAME

# Validate file existence before processing
if not FILEPATH.exists():
    raise FileNotFoundError(
        f"Dataset not found at: {FILEPATH}\n"
        f"Please verify the path and filename."
    )

logger.info(f"Target file validated: {FILEPATH}")
logger.info(f"File size: {FILEPATH.stat().st_size / (1024**2):.2f} MB")

2025-12-07 15:47:57,366 - INFO - Target file validated: D:\phd\data\LEGOS\SIT_SH_2002_2011_ENV_SnowAMSR.ease2_12500_smth25000.nc
2025-12-07 15:47:57,367 - INFO - File size: 1525.44 MB


## 1. Load Dataset and Display High-Level Summary

Using `xarray` for efficient NetCDF handling with lazy loading (memory-efficient).

In [3]:
def load_dataset(filepath: Path) -> xr.Dataset:
    """
    Load NetCDF dataset with optimal chunking for large files.
    
    Args:
        filepath: Path to the NetCDF file.
        
    Returns:
        xr.Dataset: Loaded dataset with lazy loading enabled.
        
    Raises:
        IOError: If file cannot be read.
    """
    try:
        ds = xr.open_dataset(filepath, engine="netcdf4")
        logger.info(f"Successfully loaded dataset: {filepath.name}")
        return ds
    except Exception as e:
        logger.error(f"Failed to load dataset: {e}")
        raise IOError(f"Cannot read NetCDF file: {filepath}") from e


# Load the dataset
ds = load_dataset(FILEPATH)

# Display high-level summary
print("=" * 80)
print("DATASET OVERVIEW")
print("=" * 80)
ds

2025-12-07 15:48:22,323 - INFO - Successfully loaded dataset: SIT_SH_2002_2011_ENV_SnowAMSR.ease2_12500_smth25000.nc


DATASET OVERVIEW


<xarray.Dataset> Size: 2GB
Dimensions:              (time: 55, time_bounds: 2, u: 850, v: 850)
Coordinates:
  * time                 (time) datetime64[ns] 440B 2002-10-15 ... 2011-10-15
    time_bounds          (time, time_bounds) datetime64[ns] 880B ...
    latitude             (u, v) float32 3MB ...
    longitude            (u, v) float32 3MB ...
Dimensions without coordinates: u, v
Data variables:
    snow_depth           (time, u, v) float32 159MB ...
    snow_depth_unc       (time, u, v) float32 159MB ...
    freeboard_radar      (time, u, v) float32 159MB ...
    freeboard_radar_unc  (time, u, v) float32 159MB ...
    freeboard_ice        (time, u, v) float32 159MB ...
    freeboard_ice_unc    (time, u, v) float32 159MB ...
    freeboard_tot        (time, u, v) float32 159MB ...
    freeboard_tot_unc    (time, u, v) float32 159MB ...
    SIT                  (time, u, v) float32 159MB ...
    SIT_unc              (time, u, v) float32 159MB ...
Attributes: (12/26)
    title:                Monthly Altimetric SIT products from envisat
    projection:           laea
    grid_type:            np2ease
    lat_ts:               0
    lon_0:                0
    pixel_size:           12500
    ...                   ...
    copyright_statement:  Copyright CTOH/LEGOS
    reference:            F.GARNIER, S Fleury, A.Laforge, F Remy, B.Meyssignac
    command:              cmd
    license:              https://www.aviso.altimetry.fr/fileadmin/documents/...
    references:           https://www.aviso.altimetry.fr/fileadmin/documents/...
    history:

## 2. Global Attributes Inspection

Global attributes contain critical metadata about data provenance, processing history, and conventions.

In [4]:
def display_global_attributes(ds: xr.Dataset) -> None:
    """
    Display all global attributes with formatted output.
    
    Args:
        ds: xarray Dataset to inspect.
    """
    print("=" * 80)
    print("GLOBAL ATTRIBUTES")
    print("=" * 80)
    
    if not ds.attrs:
        print("No global attributes found.")
        return
    
    for key, value in ds.attrs.items():
        # Truncate long values for readability
        value_str = str(value)
        if len(value_str) > 100:
            value_str = value_str[:100] + "..."
        print(f"  {key:<30}: {value_str}")


display_global_attributes(ds)

GLOBAL ATTRIBUTES
  title                         : Monthly Altimetric SIT products from envisat
  projection                    : laea
  grid_type                     : np2ease
  lat_ts                        : 0
  lon_0                         : 0
  pixel_size                    : 12500
  width                         : 6250000
  height                        : 6250000
  lat_0                         : -90
  lat_min                       : -90
  lat_max                       : -60
  nb_pixels_x                   : 850
  nb_pixels_y                   : 850
  filtering                     : distance
  range_filter                  : 25000
  contact                       : ctoh_products@legos.obs-mip.fr, florent.garnier@legos.obs-mip.fr, sara.fleury@legos.obs-mip.fr
  institution                   : CTOH/LEGOS/CNRS-CNES-IRD-Universite de Toulouse III
  Conventions                   : CF-1.6
  date_of_creation              : 2019-10-04 13:14:32.742473
  production                    : CT

## 3. Dimensions Analysis

Understanding the dimensional structure is critical for data manipulation and analysis.

In [5]:
def display_dimensions(ds: xr.Dataset) -> None:
    """
    Display dataset dimensions with sizes.
    
    Args:
        ds: xarray Dataset to inspect.
    """
    print("=" * 80)
    print("DIMENSIONS")
    print("=" * 80)
    print(f"{'Dimension Name':<25} {'Size':>15}")
    print("-" * 40)
    
    for dim_name, dim_size in ds.dims.items():
        print(f"  {dim_name:<25} {dim_size:>12,}")
    
    print("-" * 40)
    print(f"  {'Total dimensions:':<25} {len(ds.dims):>12}")


display_dimensions(ds)

DIMENSIONS
Dimension Name                       Size
----------------------------------------
  time                                55
  time_bounds                          2
  u                                  850
  v                                  850
----------------------------------------
  Total dimensions:                    4


C:\Users\xliu38\AppData\Local\Temp\ipykernel_17856\2578100245.py:14: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  for dim_name, dim_size in ds.dims.items():


## 4. Coordinates Inspection

Coordinates define the reference system for the data variables.

In [6]:
def display_coordinates(ds: xr.Dataset) -> None:
    """
    Display coordinate variables with their properties.
    
    Args:
        ds: xarray Dataset to inspect.
    """
    print("=" * 80)
    print("COORDINATES")
    print("=" * 80)
    print(f"{'Name':<20} {'Dtype':<12} {'Shape':<20} {'Min':<15} {'Max':<15}")
    print("-" * 82)
    
    for coord_name, coord_data in ds.coords.items():
        dtype_str = str(coord_data.dtype)
        shape_str = str(coord_data.shape)
        
        try:
            min_val = f"{float(coord_data.min().values):.4g}"
            max_val = f"{float(coord_data.max().values):.4g}"
        except (TypeError, ValueError):
            min_val = str(coord_data.values[0])[:12] if coord_data.size > 0 else "N/A"
            max_val = str(coord_data.values[-1])[:12] if coord_data.size > 0 else "N/A"
        
        print(f"  {coord_name:<20} {dtype_str:<12} {shape_str:<20} {min_val:<15} {max_val:<15}")
        
        # Display coordinate attributes
        if coord_data.attrs:
            for attr_key, attr_val in coord_data.attrs.items():
                attr_val_str = str(attr_val)[:50]
                print(f"    └─ {attr_key}: {attr_val_str}")


display_coordinates(ds)

COORDINATES
Name                 Dtype        Shape                Min             Max            
----------------------------------------------------------------------------------
  time                 datetime64[ns] (55,)                1.035e+18       1.319e+18      
    └─ standard_name: time
    └─ comment: time is fixed to the middle of the 15th of the mon
  time_bounds          datetime64[ns] (55, 2)              1.033e+18       1.32e+18       
    └─ standard_name: time_bounds
  latitude             float32      (850, 850)           -89.92          -17.84         
    └─ units: degree_north
    └─ long_name: longitude
  longitude            float32      (850, 850)           -179.9          179.9          


## 5. Data Variables Comprehensive Inspection

Detailed analysis of each data variable including dimensions, data type, shape, and attributes.

In [7]:
def display_variables(ds: xr.Dataset) -> None:
    """
    Display comprehensive information about all data variables.
    
    Args:
        ds: xarray Dataset to inspect.
    """
    print("=" * 80)
    print("DATA VARIABLES")
    print("=" * 80)
    
    for var_name, var_data in ds.data_vars.items():
        print(f"\n{'─' * 60}")
        print(f"Variable: {var_name}")
        print(f"{'─' * 60}")
        print(f"  Dimensions : {var_data.dims}")
        print(f"  Shape      : {var_data.shape}")
        print(f"  Dtype      : {var_data.dtype}")
        print(f"  Size       : {var_data.size:,} elements")
        
        # Memory footprint estimation
        memory_bytes = var_data.nbytes
        if memory_bytes >= 1024**3:
            memory_str = f"{memory_bytes / 1024**3:.2f} GB"
        elif memory_bytes >= 1024**2:
            memory_str = f"{memory_bytes / 1024**2:.2f} MB"
        elif memory_bytes >= 1024:
            memory_str = f"{memory_bytes / 1024:.2f} KB"
        else:
            memory_str = f"{memory_bytes} bytes"
        print(f"  Memory     : {memory_str}")
        
        # Attributes
        if var_data.attrs:
            print(f"  Attributes :")
            for attr_key, attr_val in var_data.attrs.items():
                attr_val_str = str(attr_val)
                if len(attr_val_str) > 60:
                    attr_val_str = attr_val_str[:60] + "..."
                print(f"    • {attr_key}: {attr_val_str}")
        else:
            print(f"  Attributes : None")


display_variables(ds)

DATA VARIABLES

────────────────────────────────────────────────────────────
Variable: snow_depth
────────────────────────────────────────────────────────────
  Dimensions : ('time', 'u', 'v')
  Shape      : (55, 850, 850)
  Dtype      : float32
  Size       : 39,737,500 elements
  Memory     : 151.59 MB
  Attributes :
    • units: m
    • long_name: snow depth from AMSR
    • comment: 

────────────────────────────────────────────────────────────
Variable: snow_depth_unc
────────────────────────────────────────────────────────────
  Dimensions : ('time', 'u', 'v')
  Shape      : (55, 850, 850)
  Dtype      : float32
  Size       : 39,737,500 elements
  Memory     : 151.59 MB
  Attributes :
    • units: m
    • long_name: snow depth uncertainties

────────────────────────────────────────────────────────────
Variable: freeboard_radar
────────────────────────────────────────────────────────────
  Dimensions : ('time', 'u', 'v')
  Shape      : (55, 850, 850)
  Dtype      : float32
  Size 

## 6. Statistical Summary of Data Variables

Quick statistical overview to validate data integrity and identify potential issues.

In [8]:
def display_statistics(ds: xr.Dataset) -> None:
    """
    Display statistical summary for numeric data variables.
    
    Args:
        ds: xarray Dataset to inspect.
    """
    print("=" * 80)
    print("STATISTICAL SUMMARY")
    print("=" * 80)
    print(f"{'Variable':<25} {'Min':>12} {'Max':>12} {'Mean':>12} {'Std':>12} {'NaN %':>10}")
    print("-" * 83)
    
    for var_name, var_data in ds.data_vars.items():
        if np.issubdtype(var_data.dtype, np.number):
            try:
                min_val = float(var_data.min().values)
                max_val = float(var_data.max().values)
                mean_val = float(var_data.mean().values)
                std_val = float(var_data.std().values)
                nan_pct = float(np.isnan(var_data.values).sum() / var_data.size * 100)
                
                print(f"  {var_name:<25} {min_val:>12.4g} {max_val:>12.4g} "
                      f"{mean_val:>12.4g} {std_val:>12.4g} {nan_pct:>9.2f}%")
            except Exception as e:
                print(f"  {var_name:<25} {'Error computing statistics':<60}")
                logger.warning(f"Could not compute statistics for {var_name}: {e}")


display_statistics(ds)

STATISTICAL SUMMARY
Variable                           Min          Max         Mean          Std      NaN %
-----------------------------------------------------------------------------------
  snow_depth                   -2.22e-18          0.6       0.1109       0.1023     86.63%
  snow_depth_unc                   0.046        0.141      0.08756      0.02851     86.63%
  freeboard_radar                -0.1427        1.172       0.0984      0.08307     87.90%
  freeboard_radar_unc                  0            2      0.03637      0.02781     88.06%
  freeboard_ice                  -0.1411        1.162       0.1241      0.08843     88.46%
  freeboard_ice_unc              0.01004        1.959       0.0417      0.02291     88.59%
  freeboard_tot                  -0.1411        1.647       0.2477       0.1644     88.46%
  freeboard_tot_unc              0.04708         1.96      0.09813      0.03307     88.59%
  SIT                             -1.156        10.28        1.331        0.889

## 7. Dataset Summary Report

Consolidated summary for documentation and reporting purposes.

In [9]:
def generate_summary_report(ds: xr.Dataset, filepath: Path) -> dict[str, Any]:
    """
    Generate a comprehensive summary report dictionary.
    
    Args:
        ds: xarray Dataset to analyze.
        filepath: Path to the source file.
        
    Returns:
        Dictionary containing summary information.
    """
    report = {
        "file_name": filepath.name,
        "file_size_mb": filepath.stat().st_size / (1024**2),
        "num_dimensions": len(ds.dims),
        "dimensions": dict(ds.dims),
        "num_coordinates": len(ds.coords),
        "coordinate_names": list(ds.coords.keys()),
        "num_data_variables": len(ds.data_vars),
        "data_variable_names": list(ds.data_vars.keys()),
        "num_global_attributes": len(ds.attrs),
        "total_memory_mb": sum(v.nbytes for v in ds.data_vars.values()) / (1024**2),
    }
    return report


# Generate and display report
report = generate_summary_report(ds, FILEPATH)

print("=" * 80)
print("SUMMARY REPORT")
print("=" * 80)
for key, value in report.items():
    if isinstance(value, float):
        print(f"  {key.replace('_', ' ').title():<30}: {value:.2f}")
    elif isinstance(value, list):
        print(f"  {key.replace('_', ' ').title():<30}: {', '.join(value)}")
    elif isinstance(value, dict):
        print(f"  {key.replace('_', ' ').title():<30}:")
        for k, v in value.items():
            print(f"    • {k}: {v:,}")
    else:
        print(f"  {key.replace('_', ' ').title():<30}: {value}")

SUMMARY REPORT
  File Name                     : SIT_SH_2002_2011_ENV_SnowAMSR.ease2_12500_smth25000.nc
  File Size Mb                  : 1525.44
  Num Dimensions                : 4
  Dimensions                    :
    • time: 55
    • time_bounds: 2
    • u: 850
    • v: 850
  Num Coordinates               : 4
  Coordinate Names              : time, time_bounds, latitude, longitude
  Num Data Variables            : 10
  Data Variable Names           : snow_depth, snow_depth_unc, freeboard_radar, freeboard_radar_unc, freeboard_ice, freeboard_ice_unc, freeboard_tot, freeboard_tot_unc, SIT, SIT_unc
  Num Global Attributes         : 26
  Total Memory Mb               : 1515.87


C:\Users\xliu38\AppData\Local\Temp\ipykernel_17856\3882977210.py:16: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  "dimensions": dict(ds.dims),


## 8. Variable Extraction and Export

Extracting key scientific variables for downstream analysis and creating a focused dataset with reduced storage footprint.

**Target Variables:**
- `time` - Temporal coordinate
- `latitude`, `longitude` - Spatial coordinates  
- `snow_depth`, `snow_depth_unc` - Snow depth and uncertainty
- `freeboard_radar`, `freeboard_radar_unc` - Radar freeboard and uncertainty
- `freeboard_ice`, `freeboard_ice_unc` - Ice freeboard and uncertainty

In [10]:
# Variable Extraction Configuration
# Following Amazon's principle of explicit configuration and Google's readability standards

VARIABLES_TO_EXTRACT: list[str] = [
    "time",
    "latitude",
    "longitude",
    "snow_depth",
    "snow_depth_unc",
    "freeboard_radar",
    "freeboard_radar_unc",
    "freeboard_ice",
    "freeboard_ice_unc",
]

# Output file naming convention following Google/Amazon standards:
# Format: {product}_{region}_{period}_{source}_{variables_summary}_{grid}_{version}.nc
OUTPUT_FILENAME: str = "legos_sh_2002_2011_env_sd_rfb_ifb_ease2_12500_v1.nc"
OUTPUT_FILEPATH: Path = DATA_DIR / OUTPUT_FILENAME

logger.info(f"Variables to extract: {VARIABLES_TO_EXTRACT}")
logger.info(f"Output file: {OUTPUT_FILEPATH}")

2025-12-07 15:59:22,066 - INFO - Variables to extract: ['time', 'latitude', 'longitude', 'snow_depth', 'snow_depth_unc', 'freeboard_radar', 'freeboard_radar_unc', 'freeboard_ice', 'freeboard_ice_unc']
2025-12-07 15:59:22,067 - INFO - Output file: D:\phd\data\LEGOS\legos_sh_2002_2011_env_sd_rfb_ifb_ease2_12500_v1.nc


In [11]:
def validate_variables(ds: xr.Dataset, variables: list[str]) -> tuple[list[str], list[str]]:
    """
    Validate that requested variables exist in the dataset.
    
    Args:
        ds: Source xarray Dataset.
        variables: List of variable names to validate.
        
    Returns:
        Tuple of (found_variables, missing_variables).
    """
    all_vars = set(ds.data_vars.keys()) | set(ds.coords.keys())
    found = [v for v in variables if v in all_vars]
    missing = [v for v in variables if v not in all_vars]
    
    return found, missing


def extract_variables(
    ds: xr.Dataset, 
    variables: list[str],
    strict: bool = True
) -> xr.Dataset:
    """
    Extract specified variables from a dataset.
    
    Args:
        ds: Source xarray Dataset.
        variables: List of variable names to extract.
        strict: If True, raise error for missing variables.
        
    Returns:
        xr.Dataset: New dataset containing only the specified variables.
        
    Raises:
        KeyError: If strict=True and variables are missing.
    """
    found, missing = validate_variables(ds, variables)
    
    if missing:
        msg = f"Missing variables in dataset: {missing}"
        if strict:
            logger.error(msg)
            raise KeyError(msg)
        else:
            logger.warning(msg)
    
    logger.info(f"Extracting {len(found)} variables: {found}")
    
    # Separate coordinates and data variables
    coord_vars = [v for v in found if v in ds.coords]
    data_vars = [v for v in found if v in ds.data_vars]
    
    # Extract data variables while preserving coordinates
    ds_extracted = ds[data_vars].copy()
    
    # Ensure requested coordinate variables are included
    for coord in coord_vars:
        if coord not in ds_extracted.coords:
            ds_extracted = ds_extracted.assign_coords({coord: ds.coords[coord]})
    
    return ds_extracted


# Validate and extract variables
found_vars, missing_vars = validate_variables(ds, VARIABLES_TO_EXTRACT)

print("=" * 80)
print("VARIABLE VALIDATION")
print("=" * 80)
print(f"  Requested variables  : {len(VARIABLES_TO_EXTRACT)}")
print(f"  Found in dataset     : {len(found_vars)}")
print(f"  Missing from dataset : {len(missing_vars)}")

if missing_vars:
    print(f"\n  ⚠ Missing variables: {missing_vars}")
    print("  Available variables in dataset:")
    for v in sorted(ds.data_vars.keys()):
        print(f"    • {v}")

if found_vars:
    print(f"\n  ✓ Variables to extract: {found_vars}")

VARIABLE VALIDATION
  Requested variables  : 9
  Found in dataset     : 9
  Missing from dataset : 0

  ✓ Variables to extract: ['time', 'latitude', 'longitude', 'snow_depth', 'snow_depth_unc', 'freeboard_radar', 'freeboard_radar_unc', 'freeboard_ice', 'freeboard_ice_unc']


In [12]:
# Extract variables (non-strict mode to handle potential naming differences)
ds_extracted = extract_variables(ds, VARIABLES_TO_EXTRACT, strict=False)

# Display extracted dataset structure
print("=" * 80)
print("EXTRACTED DATASET OVERVIEW")
print("=" * 80)
ds_extracted

2025-12-07 15:59:49,914 - INFO - Extracting 9 variables: ['time', 'latitude', 'longitude', 'snow_depth', 'snow_depth_unc', 'freeboard_radar', 'freeboard_radar_unc', 'freeboard_ice', 'freeboard_ice_unc']


EXTRACTED DATASET OVERVIEW


<xarray.Dataset> Size: 959MB
Dimensions:              (time: 55, u: 850, v: 850)
Coordinates:
  * time                 (time) datetime64[ns] 440B 2002-10-15 ... 2011-10-15
    latitude             (u, v) float32 3MB -17.84 -17.94 ... -17.94 -17.84
    longitude            (u, v) float32 3MB -45.0 -44.93 -44.86 ... 135.1 135.0
Dimensions without coordinates: u, v
Data variables:
    snow_depth           (time, u, v) float32 159MB nan nan nan ... nan nan nan
    snow_depth_unc       (time, u, v) float32 159MB nan nan nan ... nan nan nan
    freeboard_radar      (time, u, v) float32 159MB nan nan nan ... nan nan nan
    freeboard_radar_unc  (time, u, v) float32 159MB nan nan nan ... nan nan nan
    freeboard_ice        (time, u, v) float32 159MB nan nan nan ... nan nan nan
    freeboard_ice_unc    (time, u, v) float32 159MB nan nan nan ... nan nan nan
Attributes: (12/26)
    title:                Monthly Altimetric SIT products from envisat
    projection:           laea
    grid_type:            np2ease
    lat_ts:               0
    lon_0:                0
    pixel_size:           12500
    ...                   ...
    copyright_statement:  Copyright CTOH/LEGOS
    reference:            F.GARNIER, S Fleury, A.Laforge, F Remy, B.Meyssignac
    command:              cmd
    license:              https://www.aviso.altimetry.fr/fileadmin/documents/...
    references:           https://www.aviso.altimetry.fr/fileadmin/documents/...
    history:

In [13]:
def add_provenance_metadata(
    ds: xr.Dataset,
    source_file: str,
    extracted_vars: list[str],
    author: str = "Xinlong Liu"
) -> xr.Dataset:
    """
    Add provenance metadata to extracted dataset following CF conventions.
    
    Args:
        ds: Dataset to add metadata to.
        source_file: Original source filename.
        extracted_vars: List of extracted variable names.
        author: Author name for attribution.
        
    Returns:
        Dataset with updated global attributes.
    """
    from datetime import datetime, timezone
    
    ds.attrs.update({
        "title": "LEGOS Sea Ice Freeboard and Snow Depth Dataset (Extracted)",
        "institution": "University of Tasmania / LEGOS",
        "source": f"Extracted from {source_file}",
        "history": f"{datetime.now(timezone.utc).isoformat()} - "
                   f"Variables extracted: {', '.join(extracted_vars)}",
        "references": "LEGOS Sea Ice Thickness Product",
        "Conventions": "CF-1.8",
        "creator_name": author,
        "date_created": datetime.now(timezone.utc).strftime("%Y-%m-%d"),
        "processing_level": "L3 (Extracted subset)",
        "keywords": "sea ice, freeboard, snow depth, Antarctic, CryoSat-2, Envisat",
        "summary": "Subset of LEGOS SIT product containing snow depth, "
                   "radar freeboard, and ice freeboard with uncertainties.",
    })
    
    return ds


# Add provenance metadata
ds_extracted = add_provenance_metadata(
    ds_extracted,
    source_file=FILENAME,
    extracted_vars=found_vars
)

print("=" * 80)
print("UPDATED GLOBAL ATTRIBUTES")
print("=" * 80)
for key, value in ds_extracted.attrs.items():
    value_str = str(value)[:80] + "..." if len(str(value)) > 80 else str(value)
    print(f"  {key:<20}: {value_str}")

UPDATED GLOBAL ATTRIBUTES
  title               : LEGOS Sea Ice Freeboard and Snow Depth Dataset (Extracted)
  projection          : laea
  grid_type           : np2ease
  lat_ts              : 0
  lon_0               : 0
  pixel_size          : 12500
  width               : 6250000
  height              : 6250000
  lat_0               : -90
  lat_min             : -90
  lat_max             : -60
  nb_pixels_x         : 850
  nb_pixels_y         : 850
  filtering           : distance
  range_filter        : 25000
  contact             : ctoh_products@legos.obs-mip.fr, florent.garnier@legos.obs-mip.fr, sara.fleury@le...
  institution         : University of Tasmania / LEGOS
  Conventions         : CF-1.8
  date_of_creation    : 2019-10-04 13:14:32.742473
  production          : CTOH/LEGOS
  copyright_statement : Copyright CTOH/LEGOS
  reference           : F.GARNIER, S Fleury, A.Laforge, F Remy, B.Meyssignac
  command             : cmd
  license             : https://www.aviso.altimetry

In [14]:
def save_dataset(
    ds: xr.Dataset,
    filepath: Path,
    compression_level: int = 4,
    overwrite: bool = False
) -> None:
    """
    Save dataset to NetCDF with compression and validation.
    
    Args:
        ds: Dataset to save.
        filepath: Output file path.
        compression_level: zlib compression level (0-9).
        overwrite: If True, overwrite existing file.
        
    Raises:
        FileExistsError: If file exists and overwrite=False.
        IOError: If save operation fails.
    """
    if filepath.exists() and not overwrite:
        raise FileExistsError(
            f"Output file already exists: {filepath}\n"
            f"Set overwrite=True to replace."
        )
    
    # Configure compression for all variables
    encoding = {}
    for var_name in ds.data_vars:
        encoding[var_name] = {
            "zlib": True,
            "complevel": compression_level,
            "dtype": ds[var_name].dtype,
        }
    
    try:
        ds.to_netcdf(
            filepath,
            engine="netcdf4",
            encoding=encoding,
            format="NETCDF4"
        )
        
        # Validate saved file
        file_size_mb = filepath.stat().st_size / (1024**2)
        logger.info(f"Successfully saved dataset to: {filepath}")
        logger.info(f"Output file size: {file_size_mb:.2f} MB")
        
        print("=" * 80)
        print("SAVE OPERATION COMPLETE")
        print("=" * 80)
        print(f"  Output file     : {filepath.name}")
        print(f"  Output path     : {filepath.parent}")
        print(f"  File size       : {file_size_mb:.2f} MB")
        print(f"  Compression     : zlib level {compression_level}")
        print(f"  Format          : NetCDF4")
        print(f"  Status          : ✓ Success")
        
    except Exception as e:
        logger.error(f"Failed to save dataset: {e}")
        raise IOError(f"Cannot save NetCDF file: {filepath}") from e


# Save extracted dataset
save_dataset(
    ds_extracted,
    OUTPUT_FILEPATH,
    compression_level=4,
    overwrite=True  # Set to False in production to prevent accidental overwrites
)

2025-12-07 16:00:47,666 - INFO - Successfully saved dataset to: D:\phd\data\LEGOS\legos_sh_2002_2011_env_sd_rfb_ifb_ease2_12500_v1.nc
2025-12-07 16:00:47,666 - INFO - Output file size: 90.73 MB


SAVE OPERATION COMPLETE
  Output file     : legos_sh_2002_2011_env_sd_rfb_ifb_ease2_12500_v1.nc
  Output path     : D:\phd\data\LEGOS
  File size       : 90.73 MB
  Compression     : zlib level 4
  Format          : NetCDF4
  Status          : ✓ Success


In [15]:
def verify_saved_dataset(filepath: Path) -> bool:
    """
    Verify integrity of saved dataset by re-reading and validating.
    
    Args:
        filepath: Path to the saved NetCDF file.
        
    Returns:
        True if verification passes.
        
    Raises:
        AssertionError: If verification fails.
    """
    print("=" * 80)
    print("VERIFICATION")
    print("=" * 80)
    
    with xr.open_dataset(filepath, engine="netcdf4") as ds_verify:
        print(f"  File readable   : ✓")
        print(f"  Dimensions      : {dict(ds_verify.dims)}")
        print(f"  Coordinates     : {list(ds_verify.coords.keys())}")
        print(f"  Data variables  : {list(ds_verify.data_vars.keys())}")
        print(f"  Global attrs    : {len(ds_verify.attrs)} attributes")
        
        # Basic integrity checks
        assert len(ds_verify.data_vars) > 0, "No data variables found"
        assert "history" in ds_verify.attrs, "Missing provenance metadata"
        
        print(f"\n  ✓ Verification PASSED")
        
    return True


# Verify the saved dataset
verify_saved_dataset(OUTPUT_FILEPATH)

VERIFICATION
  File readable   : ✓
  Dimensions      : {'time': 55, 'u': 850, 'v': 850}
  Coordinates     : ['time', 'latitude', 'longitude']
  Data variables  : ['snow_depth', 'snow_depth_unc', 'freeboard_radar', 'freeboard_radar_unc', 'freeboard_ice', 'freeboard_ice_unc']
  Global attrs    : 32 attributes

  ✓ Verification PASSED


C:\Users\xliu38\AppData\Local\Temp\ipykernel_17856\2381443182.py:20: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"  Dimensions      : {dict(ds_verify.dims)}")


True

In [16]:
# Clean up: Close datasets to release resources
ds.close()
ds_extracted.close()
logger.info("All datasets closed successfully. Resources released.")

print("=" * 80)
print("EXTRACTION WORKFLOW COMPLETE")
print("=" * 80)
print(f"  Source file  : {FILENAME}")
print(f"  Output file  : {OUTPUT_FILENAME}")
print(f"  Location     : {DATA_DIR}")
print("=" * 80)

2025-12-07 16:01:34,636 - INFO - All datasets closed successfully. Resources released.


EXTRACTION WORKFLOW COMPLETE
  Source file  : SIT_SH_2002_2011_ENV_SnowAMSR.ease2_12500_smth25000.nc
  Output file  : legos_sh_2002_2011_env_sd_rfb_ifb_ease2_12500_v1.nc
  Location     : D:\phd\data\LEGOS


## 9. CryoSat-2 Datasets Processing

Processing two CryoSat-2 period LEGOS SIT datasets with different snow depth products:
1. **SnowAMSR** (2010-2018): Snow depth derived from AMSR-E/AMSR2
2. **SnowKaKu** (2013-2018): Snow depth derived from Ka-Ku band radar

**Objective:** Inspect structure and extract key variables for comparative analysis.

In [17]:
# CryoSat-2 Dataset Configuration
# Following Amazon's principle of explicit configuration

CS2_DATASETS: dict[str, dict[str, Any]] = {
    "cs2_amsr": {
        "filename": "SIT_SH_2010_2018_CS2_SnowAMSR.ease2_12500_smth25000.nc",
        "period": "2010_2018",
        "snow_source": "amsr",
        "description": "CryoSat-2 with AMSR-E/AMSR2 snow depth",
    },
    "cs2_kaku": {
        "filename": "SIT_SH_2013_2018_CS2_SnowKaKu.ease2_12500_smth25000.nc",
        "period": "2013_2018",
        "snow_source": "kaku",
        "description": "CryoSat-2 with Ka-Ku band radar snow depth",
    },
}

# Validate all files exist before processing
print("=" * 80)
print("CS2 DATASET VALIDATION")
print("=" * 80)

for key, config in CS2_DATASETS.items():
    filepath = DATA_DIR / config["filename"]
    config["filepath"] = filepath
    
    if filepath.exists():
        file_size_mb = filepath.stat().st_size / (1024**2)
        config["file_size_mb"] = file_size_mb
        print(f"  ✓ {key}: {config['filename']}")
        print(f"    Size: {file_size_mb:.2f} MB | Period: {config['period']}")
        logger.info(f"Validated: {filepath}")
    else:
        raise FileNotFoundError(
            f"Dataset not found: {filepath}\n"
            f"Please verify the path and filename."
        )

2025-12-07 16:10:25,665 - INFO - Validated: D:\phd\data\LEGOS\SIT_SH_2010_2018_CS2_SnowAMSR.ease2_12500_smth25000.nc
2025-12-07 16:10:25,667 - INFO - Validated: D:\phd\data\LEGOS\SIT_SH_2013_2018_CS2_SnowKaKu.ease2_12500_smth25000.nc


CS2 DATASET VALIDATION
  ✓ cs2_amsr: SIT_SH_2010_2018_CS2_SnowAMSR.ease2_12500_smth25000.nc
    Size: 1360.07 MB | Period: 2010_2018
  ✓ cs2_kaku: SIT_SH_2013_2018_CS2_SnowKaKu.ease2_12500_smth25000.nc
    Size: 1001.77 MB | Period: 2013_2018


### 9.1 Dataset Structure Inspection

Comprehensive inspection of both CryoSat-2 datasets to understand their structure and identify available variables.

In [18]:
def inspect_dataset_comprehensive(
    filepath: Path,
    dataset_id: str
) -> tuple[xr.Dataset, dict[str, Any]]:
    """
    Perform comprehensive inspection of a NetCDF dataset.
    
    Args:
        filepath: Path to the NetCDF file.
        dataset_id: Identifier for the dataset (for logging).
        
    Returns:
        Tuple of (loaded dataset, inspection report dictionary).
    """
    ds = load_dataset(filepath)
    
    report = {
        "dataset_id": dataset_id,
        "filename": filepath.name,
        "file_size_mb": filepath.stat().st_size / (1024**2),
        "dimensions": dict(ds.dims),
        "coordinates": list(ds.coords.keys()),
        "data_variables": list(ds.data_vars.keys()),
        "num_variables": len(ds.data_vars),
        "global_attributes": dict(ds.attrs),
    }
    
    return ds, report


def display_dataset_comparison(reports: list[dict[str, Any]]) -> None:
    """
    Display side-by-side comparison of dataset structures.
    
    Args:
        reports: List of inspection report dictionaries.
    """
    print("=" * 100)
    print("DATASET STRUCTURE COMPARISON")
    print("=" * 100)
    
    # Header
    headers = [r["dataset_id"] for r in reports]
    print(f"{'Property':<25}", end="")
    for h in headers:
        print(f"{h:<35}", end="")
    print()
    print("-" * 100)
    
    # File sizes
    print(f"{'File Size (MB)':<25}", end="")
    for r in reports:
        print(f"{r['file_size_mb']:<35.2f}", end="")
    print()
    
    # Number of variables
    print(f"{'Data Variables':<25}", end="")
    for r in reports:
        print(f"{r['num_variables']:<35}", end="")
    print()
    
    # Dimensions
    print(f"{'Dimensions':<25}", end="")
    for r in reports:
        dims_str = ", ".join([f"{k}:{v}" for k, v in r["dimensions"].items()])
        print(f"{dims_str:<35}", end="")
    print()


# Load and inspect both datasets
cs2_datasets: dict[str, xr.Dataset] = {}
cs2_reports: list[dict[str, Any]] = []

for key, config in CS2_DATASETS.items():
    ds, report = inspect_dataset_comprehensive(config["filepath"], key)
    cs2_datasets[key] = ds
    cs2_reports.append(report)
    config["report"] = report

display_dataset_comparison(cs2_reports)

2025-12-07 16:11:29,305 - INFO - Successfully loaded dataset: SIT_SH_2010_2018_CS2_SnowAMSR.ease2_12500_smth25000.nc
C:\Users\xliu38\AppData\Local\Temp\ipykernel_17856\242556258.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  "dimensions": dict(ds.dims),
2025-12-07 16:11:29,575 - INFO - Successfully loaded dataset: SIT_SH_2013_2018_CS2_SnowKaKu.ease2_12500_smth25000.nc


DATASET STRUCTURE COMPARISON
Property                 cs2_amsr                           cs2_kaku                           
----------------------------------------------------------------------------------------------------
File Size (MB)           1360.07                            1001.77                            
Data Variables           10                                 10                                 
Dimensions               time:49, time_bounds:2, u:850, v:850time:36, time_bounds:2, u:850, v:850


C:\Users\xliu38\AppData\Local\Temp\ipykernel_17856\242556258.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  "dimensions": dict(ds.dims),


In [19]:
# Detailed variable inspection for each dataset
for key, ds in cs2_datasets.items():
    print("\n" + "=" * 80)
    print(f"DATASET: {key.upper()}")
    print(f"File: {CS2_DATASETS[key]['filename']}")
    print("=" * 80)
    
    # Display dimensions
    print(f"\n{'─' * 40}")
    print("DIMENSIONS")
    print(f"{'─' * 40}")
    for dim_name, dim_size in ds.dims.items():
        print(f"  {dim_name:<20}: {dim_size:>10,}")
    
    # Display coordinates
    print(f"\n{'─' * 40}")
    print("COORDINATES")
    print(f"{'─' * 40}")
    for coord_name in ds.coords:
        coord = ds.coords[coord_name]
        print(f"  {coord_name:<20}: dtype={str(coord.dtype):<10} shape={coord.shape}")
    
    # Display data variables
    print(f"\n{'─' * 40}")
    print("DATA VARIABLES")
    print(f"{'─' * 40}")
    for var_name in sorted(ds.data_vars.keys()):
        var = ds.data_vars[var_name]
        print(f"  {var_name:<30}: dtype={str(var.dtype):<10} shape={var.shape}")


DATASET: CS2_AMSR
File: SIT_SH_2010_2018_CS2_SnowAMSR.ease2_12500_smth25000.nc

────────────────────────────────────────
DIMENSIONS
────────────────────────────────────────
  time                :         49
  time_bounds         :          2
  u                   :        850
  v                   :        850

────────────────────────────────────────
COORDINATES
────────────────────────────────────────
  time                : dtype=datetime64[ns] shape=(49,)
  time_bounds         : dtype=datetime64[ns] shape=(49, 2)
  latitude            : dtype=float32    shape=(850, 850)
  longitude           : dtype=float32    shape=(850, 850)

────────────────────────────────────────
DATA VARIABLES
────────────────────────────────────────
  SIT                           : dtype=float32    shape=(49, 850, 850)
  SIT_unc                       : dtype=float32    shape=(49, 850, 850)
  freeboard_ice                 : dtype=float32    shape=(49, 850, 850)
  freeboard_ice_unc             : dtype=float

C:\Users\xliu38\AppData\Local\Temp\ipykernel_17856\2454044926.py:12: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  for dim_name, dim_size in ds.dims.items():


In [20]:
# Display global attributes for each dataset
for key, ds in cs2_datasets.items():
    print("\n" + "=" * 80)
    print(f"GLOBAL ATTRIBUTES: {key.upper()}")
    print("=" * 80)
    
    if ds.attrs:
        for attr_key, attr_val in ds.attrs.items():
            attr_val_str = str(attr_val)
            if len(attr_val_str) > 70:
                attr_val_str = attr_val_str[:70] + "..."
            print(f"  {attr_key:<25}: {attr_val_str}")
    else:
        print("  No global attributes found.")


GLOBAL ATTRIBUTES: CS2_AMSR
  title                    : Monthly Altimetric SIT products from c2esaC
  projection               : laea
  grid_type                : np2ease
  lat_ts                   : 0
  lon_0                    : 0
  pixel_size               : 12500
  width                    : 6250000
  height                   : 6250000
  lat_0                    : -90
  lat_min                  : -90
  lat_max                  : -60
  nb_pixels_x              : 850
  nb_pixels_y              : 850
  filtering                : distance
  range_filter             : 25000
  contact                  : ctoh_products@legos.obs-mip.fr, florent.garnier@legos.obs-mip.fr, sara...
  institution              : CTOH/LEGOS/CNRS-CNES-IRD-Universite de Toulouse III
  Conventions              : CF-1.6
  date_of_creation         : 2019-09-10 16:02:20.388767
  production               : CTOH/LEGOS
  copyright_statement      : Copyright CTOH/LEGOS
  reference                : F.GARNIER, S Fleury, A.

### 9.2 Variable Validation and Extraction

Validating target variables exist in both datasets and extracting them with proper metadata.

In [21]:
# Target variables for extraction (same as Envisat processing)
CS2_VARIABLES_TO_EXTRACT: list[str] = [
    "time",
    "latitude",
    "longitude",
    "snow_depth",
    "snow_depth_unc",
    "freeboard_radar",
    "freeboard_radar_unc",
    "freeboard_ice",
    "freeboard_ice_unc",
]

# Output file naming convention following Google/Amazon standards
# Format: legos_sh_{period}_{satellite}_{snow_source}_sd_rfb_ifb_ease2_12500_v1.nc
CS2_OUTPUT_CONFIG: dict[str, str] = {
    "cs2_amsr": "legos_sh_2010_2018_cs2_amsr_sd_rfb_ifb_ease2_12500_v1.nc",
    "cs2_kaku": "legos_sh_2013_2018_cs2_kaku_sd_rfb_ifb_ease2_12500_v1.nc",
}

# Validate variables for each dataset
print("=" * 80)
print("VARIABLE VALIDATION FOR EXTRACTION")
print("=" * 80)

cs2_validation_results: dict[str, dict[str, list[str]]] = {}

for key, ds in cs2_datasets.items():
    found_vars, missing_vars = validate_variables(ds, CS2_VARIABLES_TO_EXTRACT)
    cs2_validation_results[key] = {
        "found": found_vars,
        "missing": missing_vars,
    }
    
    print(f"\n{key.upper()}:")
    print(f"  Requested : {len(CS2_VARIABLES_TO_EXTRACT)} variables")
    print(f"  Found     : {len(found_vars)} variables")
    print(f"  Missing   : {len(missing_vars)} variables")
    
    if missing_vars:
        print(f"  ⚠ Missing : {missing_vars}")
    else:
        print(f"  ✓ All variables available")

VARIABLE VALIDATION FOR EXTRACTION

CS2_AMSR:
  Requested : 9 variables
  Found     : 9 variables
  Missing   : 0 variables
  ✓ All variables available

CS2_KAKU:
  Requested : 9 variables
  Found     : 9 variables
  Missing   : 0 variables
  ✓ All variables available


In [22]:
def process_cs2_dataset(
    ds: xr.Dataset,
    dataset_key: str,
    config: dict[str, Any],
    variables: list[str],
    output_filename: str,
    output_dir: Path
) -> Path:
    """
    Process a single CryoSat-2 dataset: extract variables and save.
    
    Args:
        ds: Source xarray Dataset.
        dataset_key: Dataset identifier.
        config: Dataset configuration dictionary.
        variables: List of variables to extract.
        output_filename: Name for the output file.
        output_dir: Directory to save the output file.
        
    Returns:
        Path to the saved output file.
    """
    logger.info(f"Processing dataset: {dataset_key}")
    
    # Extract variables
    ds_extracted = extract_variables(ds, variables, strict=False)
    
    # Add provenance metadata
    from datetime import datetime, timezone
    
    ds_extracted.attrs.update({
        "title": f"LEGOS Sea Ice Freeboard and Snow Depth - CryoSat-2 {config['snow_source'].upper()}",
        "institution": "University of Tasmania / LEGOS",
        "source": f"Extracted from {config['filename']}",
        "history": f"{datetime.now(timezone.utc).isoformat()} - "
                   f"Variables extracted: {', '.join(variables)}",
        "references": "LEGOS Sea Ice Thickness Product",
        "Conventions": "CF-1.8",
        "creator_name": "Xinlong Liu",
        "date_created": datetime.now(timezone.utc).strftime("%Y-%m-%d"),
        "processing_level": "L3 (Extracted subset)",
        "satellite": "CryoSat-2",
        "snow_depth_source": config["snow_source"].upper(),
        "temporal_coverage": config["period"].replace("_", "-"),
        "keywords": f"sea ice, freeboard, snow depth, Antarctic, CryoSat-2, {config['snow_source']}",
        "summary": f"Subset of LEGOS CryoSat-2 SIT product ({config['description']}) "
                   f"containing snow depth, radar freeboard, and ice freeboard with uncertainties.",
    })
    
    # Save to file
    output_path = output_dir / output_filename
    save_dataset(
        ds_extracted,
        output_path,
        compression_level=4,
        overwrite=True
    )
    
    return output_path


# Process both datasets
cs2_output_paths: dict[str, Path] = {}

print("=" * 80)
print("PROCESSING CRYOSAT-2 DATASETS")
print("=" * 80)

for key, ds in cs2_datasets.items():
    print(f"\n{'─' * 60}")
    print(f"Processing: {key}")
    print(f"{'─' * 60}")
    
    output_path = process_cs2_dataset(
        ds=ds,
        dataset_key=key,
        config=CS2_DATASETS[key],
        variables=CS2_VARIABLES_TO_EXTRACT,
        output_filename=CS2_OUTPUT_CONFIG[key],
        output_dir=DATA_DIR
    )
    
    cs2_output_paths[key] = output_path

2025-12-07 16:15:02,987 - INFO - Processing dataset: cs2_amsr
2025-12-07 16:15:02,988 - INFO - Extracting 9 variables: ['time', 'latitude', 'longitude', 'snow_depth', 'snow_depth_unc', 'freeboard_radar', 'freeboard_radar_unc', 'freeboard_ice', 'freeboard_ice_unc']


PROCESSING CRYOSAT-2 DATASETS

────────────────────────────────────────────────────────────
Processing: cs2_amsr
────────────────────────────────────────────────────────────


2025-12-07 16:15:24,245 - INFO - Successfully saved dataset to: D:\phd\data\LEGOS\legos_sh_2010_2018_cs2_amsr_sd_rfb_ifb_ease2_12500_v1.nc
2025-12-07 16:15:24,261 - INFO - Output file size: 73.77 MB
2025-12-07 16:15:24,261 - INFO - Processing dataset: cs2_kaku
2025-12-07 16:15:24,261 - INFO - Extracting 9 variables: ['time', 'latitude', 'longitude', 'snow_depth', 'snow_depth_unc', 'freeboard_radar', 'freeboard_radar_unc', 'freeboard_ice', 'freeboard_ice_unc']


SAVE OPERATION COMPLETE
  Output file     : legos_sh_2010_2018_cs2_amsr_sd_rfb_ifb_ease2_12500_v1.nc
  Output path     : D:\phd\data\LEGOS
  File size       : 73.77 MB
  Compression     : zlib level 4
  Format          : NetCDF4
  Status          : ✓ Success

────────────────────────────────────────────────────────────
Processing: cs2_kaku
────────────────────────────────────────────────────────────


2025-12-07 16:15:39,979 - INFO - Successfully saved dataset to: D:\phd\data\LEGOS\legos_sh_2013_2018_cs2_kaku_sd_rfb_ifb_ease2_12500_v1.nc
2025-12-07 16:15:39,985 - INFO - Output file size: 59.95 MB


SAVE OPERATION COMPLETE
  Output file     : legos_sh_2013_2018_cs2_kaku_sd_rfb_ifb_ease2_12500_v1.nc
  Output path     : D:\phd\data\LEGOS
  File size       : 59.95 MB
  Compression     : zlib level 4
  Format          : NetCDF4
  Status          : ✓ Success


### 9.3 Verification of Extracted Datasets

Validating the integrity of saved files to ensure data quality and completeness.

In [23]:
def verify_extracted_datasets(output_paths: dict[str, Path]) -> dict[str, bool]:
    """
    Verify integrity of all extracted datasets.
    
    Args:
        output_paths: Dictionary mapping dataset keys to output file paths.
        
    Returns:
        Dictionary mapping dataset keys to verification status.
    """
    print("=" * 80)
    print("VERIFICATION OF EXTRACTED DATASETS")
    print("=" * 80)
    
    verification_results: dict[str, bool] = {}
    
    for key, filepath in output_paths.items():
        print(f"\n{'─' * 60}")
        print(f"Verifying: {key}")
        print(f"File: {filepath.name}")
        print(f"{'─' * 60}")
        
        try:
            with xr.open_dataset(filepath, engine="netcdf4") as ds_verify:
                file_size_mb = filepath.stat().st_size / (1024**2)
                
                print(f"  File readable    : ✓")
                print(f"  File size        : {file_size_mb:.2f} MB")
                print(f"  Dimensions       : {dict(ds_verify.dims)}")
                print(f"  Coordinates      : {list(ds_verify.coords.keys())}")
                print(f"  Data variables   : {list(ds_verify.data_vars.keys())}")
                print(f"  Global attrs     : {len(ds_verify.attrs)} attributes")
                
                # Integrity checks
                assert len(ds_verify.data_vars) > 0, "No data variables found"
                assert "history" in ds_verify.attrs, "Missing provenance metadata"
                assert "satellite" in ds_verify.attrs, "Missing satellite attribute"
                
                print(f"\n  ✓ Verification PASSED")
                verification_results[key] = True
                
        except Exception as e:
            logger.error(f"Verification failed for {key}: {e}")
            print(f"\n  ✗ Verification FAILED: {e}")
            verification_results[key] = False
    
    return verification_results


# Verify all extracted datasets
cs2_verification = verify_extracted_datasets(cs2_output_paths)

VERIFICATION OF EXTRACTED DATASETS

────────────────────────────────────────────────────────────
Verifying: cs2_amsr
File: legos_sh_2010_2018_cs2_amsr_sd_rfb_ifb_ease2_12500_v1.nc
────────────────────────────────────────────────────────────
  File readable    : ✓
  File size        : 73.77 MB
  Dimensions       : {'time': 49, 'u': 850, 'v': 850}
  Coordinates      : ['time', 'latitude', 'longitude']
  Data variables   : ['snow_depth', 'snow_depth_unc', 'freeboard_radar', 'freeboard_radar_unc', 'freeboard_ice', 'freeboard_ice_unc']
  Global attrs     : 35 attributes

  ✓ Verification PASSED

────────────────────────────────────────────────────────────
Verifying: cs2_kaku
File: legos_sh_2013_2018_cs2_kaku_sd_rfb_ifb_ease2_12500_v1.nc
────────────────────────────────────────────────────────────
  File readable    : ✓
  File size        : 59.95 MB
  Dimensions       : {'time': 36, 'u': 850, 'v': 850}
  Coordinates      : ['time', 'latitude', 'longitude']
  Data variables   : ['snow_depth',

C:\Users\xliu38\AppData\Local\Temp\ipykernel_17856\3951171655.py:29: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"  Dimensions       : {dict(ds_verify.dims)}")
C:\Users\xliu38\AppData\Local\Temp\ipykernel_17856\3951171655.py:29: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"  Dimensions       : {dict(ds_verify.dims)}")


In [24]:
# Statistical comparison of extracted datasets
print("=" * 80)
print("STATISTICAL COMPARISON OF EXTRACTED DATASETS")
print("=" * 80)
print(f"{'Variable':<25} {'CS2-AMSR Min':>12} {'CS2-AMSR Max':>12} "
      f"{'CS2-KaKu Min':>12} {'CS2-KaKu Max':>12}")
print("-" * 85)

# Load extracted datasets for comparison
cs2_extracted: dict[str, xr.Dataset] = {}
for key, filepath in cs2_output_paths.items():
    cs2_extracted[key] = xr.open_dataset(filepath, engine="netcdf4")

# Compare statistics for common variables
common_vars = set(cs2_extracted["cs2_amsr"].data_vars.keys()) & \
              set(cs2_extracted["cs2_kaku"].data_vars.keys())

for var_name in sorted(common_vars):
    try:
        amsr_data = cs2_extracted["cs2_amsr"][var_name]
        kaku_data = cs2_extracted["cs2_kaku"][var_name]
        
        if np.issubdtype(amsr_data.dtype, np.number):
            amsr_min = float(amsr_data.min().values)
            amsr_max = float(amsr_data.max().values)
            kaku_min = float(kaku_data.min().values)
            kaku_max = float(kaku_data.max().values)
            
            print(f"  {var_name:<25} {amsr_min:>12.4g} {amsr_max:>12.4g} "
                  f"{kaku_min:>12.4g} {kaku_max:>12.4g}")
    except Exception as e:
        logger.warning(f"Could not compare {var_name}: {e}")

# Close extracted datasets
for ds in cs2_extracted.values():
    ds.close()

STATISTICAL COMPARISON OF EXTRACTED DATASETS
Variable                  CS2-AMSR Min CS2-AMSR Max CS2-KaKu Min CS2-KaKu Max
-------------------------------------------------------------------------------------
  freeboard_ice                  -0.2935        1.092      -0.2987        1.111
  freeboard_ice_unc              0.01004        1.999            0            2
  freeboard_radar                -0.2999       0.9998      -0.2995       0.9998
  freeboard_radar_unc                  0        1.999            0        1.999
  snow_depth                  -2.313e-20          0.6      -0.8596       0.9997
  snow_depth_unc                   0.046        0.141            0        1.996
